In [1]:
import requests

def generate_puzzle():
    #asking the user for a puzzle difficulty
    diff = input('enter puzzle difficulty, easy medium or hard: ')
    while diff not in ["easy", "medium", "hard"]:
        diff = input('invalid puzzle difficulty, please select easy, medium, or hard: ')
    url = "https://sudoku-generator1.p.rapidapi.com/sudoku/generate"

    querystring = {"difficulty": diff}

    headers = {
        "X-RapidAPI-Key": "f10e533f45msh13ee4b09b80803ep113226jsn0a78991c3fd6",
        "X-RapidAPI-Host": "sudoku-generator1.p.rapidapi.com"
    }
    #getting the puzzle from the API
    response = requests.get(url, headers=headers, params=querystring)

    return response.json()

In [2]:
import copy
def convert_puzzle(puzzle): #function to convert the puzzle (puzzles are 81-character strings with periods for empty cells)
    #converts the puzzle to a list of 9 lists
    l = [list(puzzle['puzzle'][i:i+9]) for i in range(0, len(puzzle['puzzle']), 9)]
    return [[0 if i == '.' else int(i) for i in s] for s in l]


def valid(bo, num, pos): #checks to see if a particular number is a valid entry in a cell according to the rules of Sudoku
    # Check row
    for i in range(len(bo[0])):
        if bo[pos[0]][i] == num and pos[1] != i:
            return False
        

    # Check column
    for i in range(len(bo)):
        if bo[i][pos[1]] == num and pos[0] != i:
            return False

    # Check box
    box_x = pos[1] // 3
    box_y = pos[0] // 3

    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x * 3, box_x*3 + 3):
            if bo[i][j] == num and (i,j) != pos:
                return False

    return True


def print_board(bo): #prints the board in a clean format
    s = ''
    for i in range(9):
        for j in range(9):
            s += str(bo[i][j]) + ' '
            if (j + 1) % 3 == 0 and j != 8:
                s += '| '
        s += '\n'
        if (i + 1) % 3 == 0 and i < 9 - 1:
            s += '-' *21 + '\n'
    print(s)
    return
        


def find_empty(bo): #finds the first empty cell in the board
    for i in range(len(bo)):
        for j in range(len(bo[0])):
            if bo[i][j] == 0:
                return (i, j)  # row, col

    return None

def walkthrough(bo): #walks the user through the solution process for a puzzle step by step
    # the user can exit the walkthrough process at any time by pressing 's'
    print_board(bo)
    
    find = find_empty(bo)
    if not find:
        return True
    else:
        row, col = find

    user = input('press enter to continue solution, or press s then enter to stop: ')
    while user not in ['', 's']:
        user = input('Invalid input. Press enter to continue or s then enter to stop: ')
    if user == 's':
        return True
    
    for i in range(1,10):
        if valid(bo, i, (row, col)):
            bo[row][col] = i

            if walkthrough(bo):
                return True

            bo[row][col] = 0
            print('backtracking to cell {}, {}'.format(row, col))
            
    return False

def solution(bo): #solves a puzzle using the backtracking algorithm
    find = find_empty(bo)
    if not find:
        return True
    else:
        row, col = find
    
    for i in range(1,10):
        if valid(bo, i, (row, col)):
            bo[row][col] = i
            
            if solution(bo):
                return True
            
            bo[row][col] = 0
    return False

def play():
    # prompts the user to solve a puzzle on their own, by asking to enter values in one column at a time
    # if the value is incorrect, the function asks the user to repeat again
    # if the value is correct, the function updates the puzzle accordingly
    # this process continues until the puzzle is solved or the user presses 'e' to exit and see a walkthrough of the solution
    print("Welcome to Kohl's Sudoku interface!")
    p = generate_puzzle()
    board = convert_puzzle(p)
    board_copy = copy.deepcopy(board)
    solution(board_copy)
    while find_empty(board):
        print_board(board)
        quit = input('press e to exit and show solution, or press enter to continue: ')
        while quit not in ['', 'e']:
            quit = input('invalid input, press e to exit and show solution, or press enter to continue: ')
        if quit == 'e':
            return False, board
        try:
            row = int(input('enter a row number (0-8): '))
            if row >= 9:
                row = int(input('invalid row number, please enter a row number (0-8): '))
            col = int(input('enter a column number (0-8): '))
            if col >= 9:
                col = int(input('invlid column number, please enter a row number (0-8): '))
        except Exception:
            row = int(input('invalid row number, please enter a row number (0-8): '))
            col = int(input('invlid column number, please enter a row number (0-8): '))
        val = int(input('enter value to be entered into cell {}, {}: '.format(row, col)))
        while val not in [1,2,3,4,5,6,7,8,9]:
            val = int(input('invalid input, please enter value to be entered into cell {}, {}: '.format(row, col)))
        board[row][col] = val
        if board[row][col] != board_copy[row][col]:
            print('incorrect!')
            board[row][col] = 0
    return True, board


In [3]:
bo = play()
if not bo[0]:
    walkthrough(bo[1])

Welcome to Kohl's Sudoku interface!
enter puzzle difficulty, easy medium or hard: e
invalid puzzle difficulty, please select easy, medium, or hard: easy
0 0 7 | 0 9 4 | 0 0 0 
0 2 0 | 0 7 0 | 3 0 0 
6 0 3 | 0 0 0 | 7 0 8 
---------------------
5 6 0 | 8 0 0 | 0 4 7 
0 1 0 | 9 2 7 | 0 0 6 
0 0 0 | 0 0 5 | 1 8 0 
---------------------
0 8 6 | 0 3 0 | 4 1 5 
3 0 0 | 5 8 0 | 0 6 2 
1 9 5 | 0 4 6 | 0 0 0 

press e to exit and show solution, or press enter to continue: s
invalid input, press e to exit and show solution, or press enter to continue: 
enter a row number (0-8): h
invalid row number, please enter a row number (0-8): 0
invlid column number, please enter a row number (0-8): 0
enter value to be entered into cell 0, 0: 8
8 0 7 | 0 9 4 | 0 0 0 
0 2 0 | 0 7 0 | 3 0 0 
6 0 3 | 0 0 0 | 7 0 8 
---------------------
5 6 0 | 8 0 0 | 0 4 7 
0 1 0 | 9 2 7 | 0 0 6 
0 0 0 | 0 0 5 | 1 8 0 
---------------------
0 8 6 | 0 3 0 | 4 1 5 
3 0 0 | 5 8 0 | 0 6 2 
1 9 5 | 0 4 6 | 0 0 0 

press e to exit